In [1]:
import pandas as pd

df1=pd.read_csv(filepath_or_buffer= "../data/orders.csv")

df1


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [2]:
df2=pd.read_csv(filepath_or_buffer= "../data/products.csv")
df2

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [3]:
df3=pd.read_csv(filepath_or_buffer= "../data/order_products.csv")
df3

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [5]:
df1 = df1.drop(columns=['eval_set'])


In [6]:
count = df3.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
count = count.sort_values('frequency', ascending=False)[0:100].reset_index(drop = True)
count = count.merge(df2, on = 'product_id', how = 'left')
count.head(10)

,product_id,frequency,product_name,aisle_id,department_id
0,24852,472565,Banana,24,4
1,13176,379450,Bag of Organic Bananas,24,4
2,21137,264683,Organic Strawberries,24,4
3,21903,241921,Organic Baby Spinach,123,4
4,47209,213584,Organic Hass Avocado,24,4
5,47766,176815,Organic Avocado,24,4
6,47626,152657,Large Lemon,24,4
7,16797,142951,Strawberries,24,4
8,26209,140627,Limes,24,4
9,27845,137905,Organic Whole Milk,84,16


In [7]:
freq_products = list(count.product_id)
freq_products[:10]

[24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845]

In [8]:
len(freq_products)

100

In [11]:

df3 = df3[df3.product_id.isin(freq_products)]
df3.shape

(7483881, 4)

In [12]:
df3.order_id.nunique()


2351240

In [13]:
df3 = df3.merge(df2, on = 'product_id', how='left')
df3.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,28985,2,1,Michigan Organic Kale,83,4
1,2,17794,6,1,Carrots,83,4
2,3,24838,2,1,Unsweetened Almondmilk,91,16
3,3,21903,4,1,Organic Baby Spinach,123,4
4,3,46667,6,1,Organic Ginger Root,83,4


In [14]:
basket = df3.groupby(['order_id', 'product_name'])['reordered'].count().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()

product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket = basket.applymap(encode_units)
basket.head()

/var/folders/jy/scgb3j3d27gglx2_mmbhf4sr0000gn/T/ipykernel_1382/439177313.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(encode_units)


product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_items = apriori(basket, min_support=0.005, use_colnames=True, low_memory=True)
frequent_items.head()

/Users/itobuz/anaconda3/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.016150,(100% Raw Coconut Water)
1,0.025866,(100% Whole Wheat Bread)
2,0.015775,(2% Reduced Fat Milk)
3,0.036160,(Apple Honeycrisp Organic)
4,0.028616,(Asparagus)


In [17]:
from mlxtend.frequent_patterns import association_rules

# Assuming frequent_items is your frequent itemsets DataFrame
rules = association_rules(frequent_items, metric="lift", min_threshold=1, num_itemsets=10)
rules.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
172,(Lime Sparkling Water),(Sparkling Water Grapefruit),0.019796,0.032275,0.005650,0.285395,8.842637,1.0,0.005011,1.354210,0.904824,0.121706,0.261562,0.230224
173,(Sparkling Water Grapefruit),(Lime Sparkling Water),0.032275,0.019796,0.005650,0.175052,8.842637,1.0,0.005011,1.188201,0.916491,0.121706,0.158391,0.230224
148,(Bunched Cilantro),(Limes),0.019364,0.059810,0.005313,0.274347,4.586990,1.0,0.004154,1.295646,0.797434,0.071925,0.228185,0.181585
149,(Limes),(Bunched Cilantro),0.059810,0.019364,0.005313,0.088824,4.586990,1.0,0.004154,1.076230,0.831738,0.071925,0.070831,0.181585
178,(Limes),(Organic Cilantro),0.059810,0.029569,0.007471,0.124905,4.224173,1.0,0.005702,1.108943,0.811822,0.091206,0.098241,0.188776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,(Bag of Organic Bananas),(Seedless Red Grapes),0.161383,0.035168,0.005809,0.035994,1.023486,1.0,0.000133,1.000857,0.027363,0.030454,0.000856,0.100584
5,(Apple Honeycrisp Organic),(Banana),0.036160,0.200985,0.007414,0.205034,1.020144,1.0,0.000146,1.005093,0.020487,0.032272,0.005067,0.120961
4,(Banana),(Apple Honeycrisp Organic),0.200985,0.036160,0.007414,0.036888,1.020144,1.0,0.000146,1.000756,0.024713,0.032272,0.000756,0.120961
121,(Organic Granny Smith Apple),(Banana),0.024999,0.200985,0.005062,0.202504,1.007557,1.0,0.000038,1.001905,0.007693,0.022915,0.001901,0.113846


In [18]:
high_lift_conf_rules = rules[(rules['lift'] > 2) & (rules['confidence'] > 0.2)]

high_lift_conf_rules = high_lift_conf_rules.sort_values(by=['lift', 'confidence'], ascending=False)

high_lift_conf_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
172,(Lime Sparkling Water),(Sparkling Water Grapefruit),0.005650,0.285395,8.842637
148,(Bunched Cilantro),(Limes),0.005313,0.274347,4.586990
179,(Organic Cilantro),(Limes),0.007471,0.252647,4.224173
257,(Organic Garlic),(Organic Yellow Onion),0.009388,0.201069,4.168026
305,(Raspberries),(Strawberries),0.005149,0.210028,3.454510
312,"(Bag of Organic Bananas, Organic Strawberries)",(Organic Hass Avocado),0.006408,0.244467,2.691213
264,(Organic Lemon),(Organic Hass Avocado),0.009036,0.242131,2.665496
306,"(Bag of Organic Bananas, Organic Baby Spinach)",(Organic Hass Avocado),0.005187,0.242119,2.665364
247,(Organic Cucumber),(Organic Hass Avocado),0.007424,0.217136,2.390343
314,"(Organic Strawberries, Organic Hass Avocado)",(Bag of Organic Bananas),0.006408,0.369319,2.288464


#### 